In [1]:
!uv add torch

Resolved 29 packages in 832ms
 Downloaded setuptools
 Downloaded networkx
 Downloaded sympy
 Downloaded torch
Prepared 8 packages in 1m 46s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 10 packages in 8.33s
 + filelock==3.20.3
 + fsspec==2026.1.0
 + jinja2==3.1.6
 + markupsafe==3.0.3
 + mpmath==1.3.0
 + networkx==3.6.1
 + setuptools==80.10.2
 + sympy==1.14.0
 + torch==2.10.0
 + typing-extensions==4.15.0


In [6]:
import torch
print(torch.cuda.is_available())  # Should return True
# Should print your GPU name


False


In [ ]:
import torch
import math
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
print(device)



cpu


**Introduction**

1. Firstly we shall know the basics of logistic regression

2. It primarily uses binary cross entropy as its loss function

3. Here only w is set to requires gradient true, so its tracked to the auto grad tree to caluclate gradient

4. Now here we are manually implementing the Adam optimizer function which works withh momentum and variance, it basically works with the moving averages of the gradients of the weigts

5. Unlike Stochastic Gradient Descent optimizer we make sure we take the consideration of how fast or slow the gradient descant must take place

6. It uses the first moment (momentum) and the second moment (uncentered variance) of the gradients to scale the updates.

7. It follows the same for lr, learning rate, with adam optim we can change learning rate for everystep

8. Here we just used decay to make sure its changed after 5 consecutive iterations where loss is not being decreased then we mutiply decay to get new lr, this is nothing but ReduceLR on Plateu ... This is generally done for validation loss

9. So, in training we do adam for w, lr etc. in validation time we use reducelrplateu generally

In [ ]:
X = torch.tensor([1.0,2.0,3.0,4.0],device=device)

Y = torch.tensor([2.0,3.0,4.0,5.0],device=device)

w = torch.tensor(0.0,requires_grad=True,device=device)

b = torch.tensor(0.0,requires_grad=True,device =device)

lr = 0.05
beta1, beta2 = 0.9, 0.999
epsilon = 1e-8

# --- SCHEDULER SETTINGS ---
best_loss = float('inf')
patience = 5          # How many epochs to wait before dropping LR
patience_counter = 0
decay_factor = 0.5    # Cut LR in half



In [ ]:

m, v, t = 0, 0, 0
for epoch in range(100):

  y_new = 1/(1 + torch.exp(w*X + b))

  loss = -(Y*torch.log(y_new) + (1-Y)*torch.log(1-y_new)).mean()

  loss.backward()
  with torch.no_grad():
        t += 1
        g = w.grad
        m = beta1 * m + (1 - beta1) * g
        v = beta2 * v + (1 - beta2) * (g**2)
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)

        w -= lr * m_hat / (torch.sqrt(v_hat) + epsilon)


  w.grad.zero_()
  if loss.item() < best_loss:

        best_loss = loss.item()

        patience_counter = 0

  else:
    patience_counter +=1

  if patience_counter >=patience:
    lr = lr*decay_factor

    patience_counter = 0

  if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss {loss.item():.6f}, w {w.item():.4f}, current_lr {lr}")

print(f"\nFinal w: {w.item():.4f}")